In [1]:
#import library for training model
import pandas as pd
import tensorflow as tf

Reference Link: https://towardsdatascience.com/performing-classification-in-tensorflow-95368fde289c

In [2]:
#downloading dataset from file
df = pd.read_csv('/home/raikmitl-5/Desktop/ENV_Project/Dataset/dataset_depth.csv', index_col='Unnamed: 0')

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1506 entries, 0 to 249
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Chin         1506 non-null   float64
 1   Nose         1506 non-null   float64
 2   RightEye1    1506 non-null   float64
 3   RightEye2    1506 non-null   float64
 4   LeftEye1     1506 non-null   float64
 5   LeftEye2     1506 non-null   float64
 6   MouthRight   1506 non-null   float64
 7   MouthTop     1506 non-null   float64
 8   MouthLeft    1506 non-null   float64
 9   MouthBottom  1506 non-null   float64
 10  Label        1506 non-null   object 
dtypes: float64(10), object(1)
memory usage: 141.2+ KB
None


In [4]:
df['Label'].unique()

array(['Photo', 'Real_Face'], dtype=object)

In [5]:
ls_label = []
#change the label: Photo=0, Real_face=1
for i in df['Label']:
    if i == 'Photo':
        ls_label.append(0)
    if i == 'Real_Face':
        ls_label.append(1)

df['Label'] = ls_label

In [6]:
print(df)

               Chin     Nose  RightEye1  RightEye2  LeftEye1  LeftEye2  \
Unnamed: 0                                                               
0             164.0    168.0      166.0      167.0     172.0     172.0   
1           21903.0  11151.0      244.0    10759.0    9292.0   10221.0   
2             541.0    670.0      188.0      188.0   17522.0   19165.0   
3             206.0    205.0      205.0      205.0     567.0     573.0   
4             226.0    232.0      235.0      235.0     538.0     543.0   
...             ...      ...        ...        ...       ...       ...   
245           584.0    480.0      480.0      475.0     520.0   13049.0   
246           625.0    509.0      531.0      517.0     545.0     554.0   
247           556.0    454.0      509.0      484.0     490.0     499.0   
248           577.0    458.0      495.0      480.0     513.0     541.0   
249           550.0    424.0      501.0      449.0     439.0     447.0   

            MouthRight  MouthTop  Mou

In [7]:
#Extracting the columns that are numerical in nature
cols_data = ['Chin','Nose','RightEye1','RightEye2','LeftEye1',
             'LeftEye2','MouthRight','MouthTop','MouthLeft','MouthBottom']

#Using Lambda function to normalize our features
df[cols_data] = df[cols_data].apply(lambda x: (x-x.min())/(x.max()-x.min()))

In [8]:
#Creating feature columns from numerical data
chin_ft = tf.feature_column.numeric_column("Chin")
nose_ft = tf.feature_column.numeric_column("Nose")
eyeR1_ft = tf.feature_column.numeric_column("RightEye1")
eyeR2_ft = tf.feature_column.numeric_column("RightEye2")
eyeL1_ft = tf.feature_column.numeric_column("LeftEye1")
eyeL2_ft = tf.feature_column.numeric_column("LeftEye2")
mouthR_ft = tf.feature_column.numeric_column("MouthRight")
mouthL_ft = tf.feature_column.numeric_column("MouthLeft")
mouthT_ft = tf.feature_column.numeric_column("MouthTop")
mouthB_ft = tf.feature_column.numeric_column("MouthBottom")

In [9]:
feature_cols = [chin_ft, nose_ft, eyeR1_ft, eyeR2_ft, eyeL1_ft, eyeL2_ft, 
                mouthR_ft, mouthT_ft, mouthL_ft, mouthB_ft]

In [10]:
#Spliting the features and label
labels = df['Label']
X_data = df.drop('Label', axis=1)

In [11]:
#Importing the module for performing train/test split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_data,labels,
                                                    test_size=0.03,
                                                    random_state=101)

In [12]:
#Defining the input function
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_train,
                                                           y=y_train,
                                                           batch_size=10,
                                                           num_epochs=1000,
                                                           shuffle=True)

In [13]:
#Defining the Linear Classifier
model = tf.compat.v1.estimator.LinearClassifier(feature_columns=feature_cols,
                                                n_classes=2,
                                                model_dir='/home/raikmitl-5/Desktop/ENV_Project/Checkpoint_dir')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/raikmitl-5/Desktop/ENV_Project/Checkpoint_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
model.train(input_fn=input_func,steps=1000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.


/home/raikmitl-5/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /home/raikmitl-5/Desktop/ENV_Project/Checkpoint_dir/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 6.931472, step = 0
INFO:tensorflow:global_step/sec: 543.406
INFO:tensorflow:loss = 6.1664877, step = 100 (0.186 sec)
INFO:tensorflow:global_step/sec: 834.932
INFO:tensorflow:loss = 5.008794, step = 200 (0.119 sec)
INFO:tensorflow:global_step/sec: 891.787
INFO:tensorflow:loss = 5.5263267, step = 300 (0.111 sec)
INFO:tensorflow:global_step/sec: 955.533
INFO:tensorflow:loss = 5.725009, step = 400 (0.105 sec)
INFO:tensorflow:global_

In [15]:
pred_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_test,
                                                                batch_size=10,
                                                                num_epochs=1,
                                                                shuffle=False)

In [17]:
predictions = list(model.predict(input_fn=pred_input_func))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raikmitl-5/Desktop/ENV_Project/Checkpoint_dir/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [18]:
predictions[0]

{'logits': array([0.17457801], dtype=float32),
 'logistic': array([0.543534], dtype=float32),
 'probabilities': array([0.456466, 0.543534], dtype=float32),
 'class_ids': array([1]),
 'classes': array([b'1'], dtype=object),
 'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object)}

In [19]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [20]:
final_preds[:10]

[1, 1, 0, 0, 0, 0, 0, 1, 0, 1]

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.75      0.78      0.76        27
           1       0.67      0.63      0.65        19

    accuracy                           0.72        46
   macro avg       0.71      0.70      0.71        46
weighted avg       0.72      0.72      0.72        46



Reference Link: https://medium.com/analytics-vidhya/save-and-load-a-tensorflow-estimator-model-for-predictions-233b798620a9

In [22]:
#save the model for Python
import os
import shutil

inputFn = tf.compat.v1.estimator.export.build_parsing_serving_input_receiver_fn(
    tf.feature_column.make_parse_example_spec(feature_cols))

Output_dir = '/home/raikmitl-5/Desktop/ENV_Project/Checkpoint_dir/Model_dir'
shutil.rmtree(Output_dir, ignore_errors=True)
modelBasePath = os.path.join(Output_dir, "model")

modelPath = model.export_saved_model(modelBasePath, inputFn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None


/home/raikmitl-5/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Restoring parameters from /home/raikmitl-5/Desktop/ENV_Project/Checkpoint_dir/model.ckpt-1000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/raikmitl-5/Desktop/ENV_Project/Checkpoint_dir/Model_dir/model/temp-1617948643/saved_model.pb
